In [68]:
# Librerias y Dependencias
# =================================================
import sys
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics import mean_absolute_error,max_error,mean_squared_error,r2_score,accuracy_score

In [2]:
# Funcion para Normalizar la Vista minable a exepción de la etiqueta(Variable Objetivo)
# df: es la matriz df.values [] , no incluye la etiqueta del grupo
def NormalizeViewMinable(df,valMin, dataRange):
    dataset_normalizado = np.empty((df.shape[0], df.shape[1]))
    for i in range(df.shape[0]):
        for j in range(df.shape[1]):
            dataset_normalizado[i][j]= (df[i][j] - valMin[j])/dataRange[j]

    return dataset_normalizado





def predictionTest(dataset_test_norm, df_norm, vector_pesos_optmizacion, final_datset_join):
    df_groups_finally = df_norm.copy()
    #minDep = sys.float_info.max
    lista_grupos = []
    print("Longitud dataset norm: ",dataset_test_norm.shape[0])
    print("Vista minable : ",df_groups_finally.shape[0])
    for z in range(int(dataset_test_norm.shape[0])):
        #print("Registro de Validación : ", z)
        minDep = sys.float_info.max
        posMinDep = 0
        #print("Minima Distancia: ", minDep)
        for k in range(int(df_groups_finally.shape[0])):
            ri = vector_pesos_optmizacion * np.power((dataset_test_norm[z]- df_groups_finally[k]),2) 
            dE = np.sum(ri)
            if dE < minDep:
                posMinDep = k
                minDep=dE
        
        #print("zz, " , z)
        # Grupo seleccionado
        print(f"Minima Distancia, registro {z} ", minDep)
        print("Posicion: ", posMinDep)
        grupoSelected = int(final_datset_join.values[posMinDep][-1]) 
        lista_grupos.append(grupoSelected)
        print(grupoSelected)
        
    
    return lista_grupos



def metricasPerformanceCLR(y_true, yhat):

    R2 = r2_score(y_true, yhat)
    MAE =  mean_absolute_error(y_true, yhat)
    ME = max_error(y_true, yhat)
    MSE = mean_squared_error(y_true, yhat)
    print("Coeficiente de determinación : ", R2)
    print("Error absoluto medio: ", MAE)
    print("Maximo error residual: ", ME)
    print("RMSE: ", MSE)


In [58]:
# 1. Conjunto de Test
dataset_test = pd.read_csv("FASE2/dataset_test.csv")

# 2. Codificadores - Normalización 
valMin = np.loadtxt("FASE2/Encoder_ValMin.txt")
dataRange = np.loadtxt("FASE2/Encoder_dataRange.txt")


# 3. Vector de pesos mejor ajuste
#vp = np.loadtxt("FASE2/bestFitnes.txt")
#vector_pesos = vp.flatten()

vp = np.loadtxt("FASE2/bestSA.txt")
vector_pesos = vp.flatten()
VP = vector_pesos[0:174]

# 4. Vista minable de grupos de calidad 
final_dataset_join = pd.read_csv("FASE2/Final_dataset_join.csv")
dataset_validation = dataset_test.copy()
dataset_validation = dataset_validation.drop(["ID_LOTE", "RDT_AJUSTADO"], axis=1)
dataset_validation.head(5)

# 5. cargue de los modelos
lista_modelos = []
for i in range(3):
    model = joblib.load(f'FASE2/modelo_entrenado_{i}.pkl') # Carga del modelo.
    lista_modelos.append(model)


print(len(lista_modelos))


3


In [63]:
sum(VP)

1.0003999999999997

In [59]:
#2.Normalizacion dataset | originaL con agrupaciones
df_norm = NormalizeViewMinable(final_dataset_join.values[:,:-1],valMin, dataRange)
print(df_norm.shape)


# Normalización Conjunto de test
dataset_test_norm = NormalizeViewMinable(dataset_validation.values[:,:], valMin, dataRange)
print(dataset_test_norm.shape)


(759, 174)
(40, 174)


In [60]:
# Predicciones para c/d grupo

for m in range(len(lista_modelos)):
    print(f"Predicciones modelo {m}")
    psi_pred = []
    for r in  range(len(dataset_validation)):
        psi_predicho = lista_modelos[m].predict(dataset_validation.values[r].reshape(1,-1))
        psi_pred.append(psi_predicho[0])

    
    np.savetxt(f'ResultadosImprovisacion/Comparaciones/predict_model_{m}.csv',psi_pred, delimiter=',')
    psi_pred = []

Predicciones modelo 0
Predicciones modelo 1
Predicciones modelo 2


In [61]:
y_true = dataset_test.RDT_AJUSTADO
np.savetxt(f'ResultadosImprovisacion/Comparaciones/y_true.csv',y_true, delimiter=',')

In [62]:
#Lista de 
Grupos = predictionTest(dataset_test_norm, df_norm, VP, final_dataset_join)
np.savetxt(f'ResultadosImprovisacion/Comparaciones/temp.csv',Grupos, delimiter=',')

Longitud dataset norm:  40
Vista minable :  759
Minima Distancia, registro 0  0.005921243123010041
Posicion:  5
0
Minima Distancia, registro 1  0.0022404957196595003
Posicion:  255
0
Minima Distancia, registro 2  9.867203814801255e-05
Posicion:  6
0
Minima Distancia, registro 3  0.0010228397341060463
Posicion:  53
0
Minima Distancia, registro 4  0.014929225078573829
Posicion:  475
1
Minima Distancia, registro 5  0.004299884595837439
Posicion:  401
1
Minima Distancia, registro 6  0.002667689953011719
Posicion:  193
0
Minima Distancia, registro 7  0.00308369163128942
Posicion:  223
0
Minima Distancia, registro 8  0.008452360037730918
Posicion:  412
1
Minima Distancia, registro 9  0.012266240173246727
Posicion:  451
1
Minima Distancia, registro 10  0.003987062389285598
Posicion:  740
2
Minima Distancia, registro 11  0.010565688245439594
Posicion:  364
1
Minima Distancia, registro 12  0.006486915686620368
Posicion:  501
1
Minima Distancia, registro 13  0.00022426813320288824
Posicion:  54


In [65]:
yreal =[0,
1,
0,
1,
1,
1,
0,
0,
1,
2,
2,
2,
2,
2,
2,
1,
2,
2,
0,
2,
2,
1,
2,
1,
2,
0,
2,
1,
1,
2,
0,
2,
1,
0,
2,
2,
2,
1,
0,
0]


In [66]:
y_sa = [0,0,0,0,1,1,0,0,1,1,2,1,1,0,2,1,2,2,1,2,1,2,2,0,1,1,1,0,2,2,2,2,2,0,2,0,1,1,1,1]


In [69]:
accuracy_score(yreal,y_sa)

0.475